In [5]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np


In [6]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\ESTADIC")

In [7]:
df18=pd.read_csv("ESTADIC_educacao_2018.csv", sep=';')
df21=pd.read_csv("ESTADIC_educacao_2021.csv", sep=';')

In [8]:
df18 = df18[['Cod UF', 'uf', 'genero', 'idade', 'cor_raca',
        'escolaridade']]
df21 = df21[['Cod UF', 'uf', 'genero', 'idade', 'cor_raca',
        'escolaridade']]
        

In [9]:
df18['ano']=2018
df21['ano']=2021

In [10]:
df = pd.concat([df18,df21])

In [11]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [12]:
x = df.pivot_table(columns=['ano','genero', 'faixa_etaria', 'cor_raca',
        'escolaridade'], aggfunc='size')
y = pd.DataFrame(x)
y = y.reset_index() ##inserir a contagem como coluna
y = y.rename(columns={0:'count'})

In [13]:
df = y[y['count']>0]

In [16]:
df['cor_raca'].unique()

array(['Branca', 'Parda', 'Indígena', 'Preta'], dtype=object)

In [15]:
df['cor_raca'] = np.where(df['cor_raca']=='Pardo','Parda',df['cor_raca'])

In [ ]:
### Subindo para o GBQ

In [3]:
client = bigquery.Client()

In [17]:
dataset_ref = client.dataset('cargos_lideranca')

In [18]:
df=df.rename(columns={'count':'quantidade'})

In [19]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
 bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
 bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação'),
 bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
 bigquery.SchemaField('escolaridade','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação'),
 bigquery.SchemaField('quantidade','INTEGER',description='Área deformação.')
 ]

In [20]:
table_ref = dataset_ref.table('ESTADIC_perfil_gestor_educacao_consolidado')

In [21]:
job_config = bigquery.LoadJobConfig(schema=schema)

In [22]:
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=de1941e8-6cea-4abf-871b-64ff6cf23f8a>

In [4]:
## Deletar table 

client.delete_table('repositoriodedadosgpsp.cargos_lideranca.ESTADIC_perfil_gestor_educacao_consolidado', not_found_ok=True)  